In [2]:
import csv
import json
import sqlite3
import pandas as pd
import util
import ast
from util import connect_to_preprocess_db, connect_to_basket_db


In [14]:

# Connect to the database
conn = sqlite3.connect('tipitaka_db.db')
cursor = conn.cursor()

# Read the SQL script file
with open('tipitaka_schema.sql', 'r') as file:
    script = file.read()

# Execute the SQL script
cursor.executescript(script)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [11]:
conn = sqlite3.connect('/Users/janekim/Developer/tipitaka_db_dev/tipitaka_db.db')
query = "SELECT count(*) FROM translation"
df = pd.read_sql_query(query, conn)
df

,count(*)
0,73422


In [ ]:
# Connect to preprocess.db to get translations data
conn = connect_to_preprocess_db()
predb_translations_df = pd.read_sql_query(f"SELECT * FROM sutta_translations_json", conn)

# Ingest from predb_translations_df
translations_df = predb_translations_df[['id', 'uid', 'lang', 'author_uid']]

# Get actual text from Github/sc_data
html_text_df = pd.read_sql_query("SELECT * FROM html_text", conn)
sc_bilara_texts_df = pd.read_sql_query("SELECT * FROM sc_bilara_texts", conn)
conn.close()

# Merge html_text_df with translations_df
merged_df = translations_df.merge(html_text_df[['_key', 'file_path']], left_on='id',
                                    right_on='_key', how='left')
# Merge sc_bilara_texts with merged_df
merged_df = merged_df.merge(sc_bilara_texts_df[['_key', 'file_path']], left_on='id',
                            right_on='_key', how='left',
                            suffixes=('_html', '_sc_bilara'))

# Drop _key_html and _key_sc_bilara since they are the same as `id`
merged_df = merged_df.drop(columns=['_key_html', '_key_sc_bilara'])
# Combine file_path_html and file_path_sc_bilara into file_path
merged_df['file_path'] = merged_df['file_path_html'].combine_first(merged_df['file_path_sc_bilara'])
merged_df = merged_df.drop(columns=['file_path_html', 'file_path_sc_bilara'])
merged_df['file_path'] = merged_df['file_path'].str.replace('^/opt/sc/sc-flask/', '')

# Note the suttas which don't have a corresponding text/translation
missing_text = merged_df[merged_df['file_path'].isnull()]
missing_text.to_csv(f'sutta_missing_text.csv', index=False)

# Populate `text` column using `file_path`
def get_file_contents(file_path: str):
    if pd.isna(file_path):
        return None
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            if '.json' in file_path:
                return json.load(file)
            return file.read()
    except FileNotFoundError:
        print(f'File not found: {file_path}')
        return None
            
merged_df['text'] = merged_df['file_path'].apply(get_file_contents)